In [ ]:
!pip install -qU /content/drive/MyDrive/Final_Projects
!pip install -qU langchain-groq
!pip install -q flask flask-cors
!pip install -q pyngrok
!pip install -q flask-ngrok

In [ ]:
!apt-get update
!apt-get install -y tesseract-ocr

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Final_Projects')

from data_manager import DataManager

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferMemory
from langchain_groq import ChatGroq
from data_manager import DataManager
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok
import os
import json

import os
os.environ['GROQ_API_KEY'] = Ues api key

os.environ['lamaaccess'] = use api key

# Ensure GROQ_API_KEY is set
if 'GROQ_API_KEY' not in os.environ:
    raise EnvironmentError("GROQ_API_KEY is not set. Please set it as an environment variable.")

# Ngrok authentication token
ngrok.set_auth_token(use api key)

# Initialize Flask app
app = Flask(__name__)
CORS(app, resources={r"/*": {"origins": "*"}})
#run_with_ngrok(app)

# Paths and setup
folder_path="/content/drive/MyDrive/Final_Project1/my_data"
#folder_path = "/content/drive/MyDrive/Final_Projects/Cyber_Securitydataset"
persist_dir = "/content/drive/MyDrive/Final_Project/Chromdb"
responses_path = "/content/drive/MyDrive/Final_Project/responses.json"
urls = [
    "https://www.geeksforgeeks.org/cyber-security-types-and-importance/",
    "https://en.wikipedia.org/wiki/Computer_security",
]

# Initialize DataManager
data_manager = DataManager(persist_dir)
try:

    # Extract and clean data
    folder_data = data_manager.extract_data_from_folder(folder_path)
    website_data = data_manager.extract_data_from_websites(urls)
    combined_data = folder_data + website_data
    # Chunk and embed data
    chunks = data_manager.chunk_data(combined_data)
    data_manager.embed_and_store_data(chunks)
    retrieval_chain = data_manager.get_retrieval_chain()
except Exception as e:
    print(f"Error initializing DataManager: {e}")
    raise

# Initialize ChatGroq model
try:
    chat_model = ChatGroq(
        temperature=0,
        groq_api_key=os.environ['GROQ_API_KEY'],
        model="llama3-8b-8192"
    )
except Exception as e:
    print(f"Error initializing ChatGroq: {e}")
    raise

# Initialize memory for context retention
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Create the RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=chat_model,
    chain_type="stuff",
    retriever=retrieval_chain,
    return_source_documents=True,
)

# Function to save responses
def save_responses_to_file(response, file_path):
    try:
        if not os.path.exists(file_path):
            with open(file_path, "w") as file:
                json.dump([], file)

        with open(file_path, "r+") as file:
            responses = json.load(file)
            responses.append(response)
            file.seek(0)
            json.dump(responses, file, indent=4)
    except Exception as e:
        print(f"Error saving responses to file: {e}")

# Function to process responses
def process_llm_response(query, llm_response, return_sources=False):
    try:
        result = llm_response['result']
        sources = [
            source.metadata.get('source', 'Unknown') for source in llm_response.get("source_documents", [])
        ]

        response_entry = {
            "query": query,
            "result": result,
            "sources": sources
        }
        save_responses_to_file(response_entry, responses_path)

        if return_sources:
            return result, sources
        return result
    except Exception as e:
        print(f"Error processing LLM response: {e}")
        return "An error occurred while processing the response."

# Flask API endpoint
@app.route('/api/query', methods=['POST'])
def query():
    try:
        data = request.json
        question = data.get("question", "")

        # Ensure memory manually retains context
        memory.chat_memory.add_user_message(question)

        # Invoke the QA chain
        llm_response = qa_chain.invoke({"query": question})

        # Add AI response to memory for context retention
        memory.chat_memory.add_ai_message(llm_response['result'])

        # Process the response
        response = process_llm_response(question, llm_response)
        return jsonify({"answer": response})
    except Exception as e:
        print(f"Error in /api/query endpoint: {e}")
        return jsonify({"error": "Internal Server Error"}), 500

@app.route('/api/sources', methods=['POST'])
def sources():
    try:
        data = request.json
        question = data.get("question", "")

        # Ensure memory manually retains context
        memory.chat_memory.add_user_message(question)

        # Invoke the QA chain
        llm_response = qa_chain.invoke({"query": question})

        # Add AI response to memory for context retention
        memory.chat_memory.add_ai_message(llm_response['result'])

        # Process the response and return sources
        _, sources = process_llm_response(question, llm_response, return_sources=True)
        return jsonify({"sources": sources})
    except Exception as e:
        print(f"Error in /api/sources endpoint: {e}")
        return jsonify({"error": "Internal Server Error"}), 500

# Run Flask app with ngrok tunnel
if __name__ == "__main__":
    try:
        # Start ngrok tunnel
        public_url = ngrok.connect(5000)
        print(f"Ngrok Tunnel URL: {public_url}")

        # Start Flask server
        app.run(host="0.0.0.0", port=5000)
    except Exception as e:
        print(f"Error starting Flask app with ngrok: {e}")

<ipython-input-10-f7ca35fa36c8>:67: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


Ngrok Tunnel URL: NgrokTunnel: "https://ab94-34-125-165-121.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [11/Dec/2024 07:26:51] "OPTIONS /api/query HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Dec/2024 07:26:52] "POST /api/query HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Dec/2024 07:27:14] "OPTIONS /api/query HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Dec/2024 07:27:15] "POST /api/query HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Dec/2024 07:27:35] "OPTIONS /api/query HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Dec/2024 07:27:36] "POST /api/query HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Dec/2024 07:27:50] "OPTIONS /api/query HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Dec/2024 07:27:51] "POST /api/query HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Dec/